In [1]:
# 1. The "Nuclear" Install (Force Sync Versions)
%%capture
# Uninstall to be absolutely sure
!pip uninstall -y unsloth unsloth_zoo

# Install BOTH from source so they speak the same language
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# Install other dependencies
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# 2. Import libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 3. Load the Llama-3 Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [2]:
# 4. Add LoRA Adapters (The "Fine-Tuning" Setup)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank: The "complexity" of the new skills we're teaching.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [3]:
# 5. Prepare the Data
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Load dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [4]:
# 6. Train the Model
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 60 steps for a quick demo
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

print("Starting Training...")
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/18612 [00:00<?, ? examples/s]

Starting Training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


...

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.586800
2,1.588000
3,1.233200
4,1.356100
5,1.247400
6,1.104900
7,0.853400
8,0.747800
9,0.979300
10,0.536400


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▁▃▂▂▂▃▃▁▂█▃▂▃▂▂▃▂▁▃▃▁▁▁▁▁▂▁▁▂▁▁▂▁▁▂▂▁▂▁▁
train/learning_rate,▁▂▄▅▇██▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁
train/loss,██▇▆▃▁▃▃▁▂▂▂▄▃▂▄▂▂▂▂▂▁▂▃▁▂▂▃▂▁▁▁▂▂▁▁▂▂▂▂
total_flos,5419617016283136.0
train/epoch,0.02579
train/global_step,60
train/grad_norm,0.33782
train/learning_rate,0.0
train/loss,0.5253


TrainOutput(global_step=60, training_loss=0.704929328461488, metrics={'train_runtime': 634.5778, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.095, 'total_flos': 5419617016283136.0, 'train_loss': 0.704929328461488, 'epoch': 0.025789813023855575})

In [5]:
# 7. Test it (Inference)
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Fix the python code below.", # Instruction
        "def add(a, b): return a - b", # Input (Broken code)
        "", # Output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print("\n\n### MODEL GENERATED OUTPUT ###\n")
print(tokenizer.batch_decode(outputs)[0])



### MODEL GENERATED OUTPUT ###

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Fix the python code below.

### Input:
def add(a, b): return a - b

### Response:
def add(a, b): return a + b<|end_of_text|>


In [7]:
# Save the LoRA adapters to Google Drive
from google.colab import drive
drive.mount('/content/drive')

model.save_pretrained("/content/drive/MyDrive/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/lora_model")
print("Model saved to Google Drive!")

Mounted at /content/drive
Model saved to Google Drive!
